In [14]:
import os
import copy

import rospy
import rospkg
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
from rosbag import Bag
from pathlib import Path
from cv_bridge import CvBridge

# The following import is temporary for coding purpose, change if there is error
from rosbag_read_utils.Rosbag_Reader import RosbagReader


class FormatConverter:
    def __init__(self) -> None:
        pass


def read_observations(obs_topic: str, rosbag: Bag):
    obs = pd.DataFrame(columns=["timestamp"] + [f"joint_{i}" for i in range(7)])

    # initialise progress bar
    pbar = tqdm(total=rosbag.get_message_count())

    for topic, msg, ts in rosbag.read_messages(obs_topic):
        pbar.update(1)
        timestamp = pd.to_datetime(ts.to_nsec(), unit="ns")
        obs.loc[len(obs)] = [timestamp] + [msg.position[0],
                                           msg.position[1],
                                           msg.position[2],
                                           msg.position[3],
                                           msg.position[4],
                                           msg.position[5],
                                           msg.position[6]] 
    # return the first 5 rows
    obs.head()

    return obs

def read_images(img_topic: str, rosbag: Bag) -> pd.DataFrame:
    images = pd.DataFrame(columns=["timestamp", "cv::Mat"])

    bridge = CvBridge()

    pbar = tqdm(total=rosbag.get_message_count(img_topic))

    for topic, msg, ts in rosbag.read_messages(img_topic):
        pbar.update(1)
        timestamp = pd.to_datetime(ts.to_sec(), unit='s')
        cv_image_msg = bridge.imgmsg_to_cv2(msg, desired_encoding="passthrough")
        images.loc[len(images)] = [timestamp, cv_image_msg]

    return images
    
def read_and_sync_image(obs: pd.DataFrame ,img_dir: str, img_topic: str, rosbag: Bag, bag_reader: RosbagReader):
    cv_image = []
    pbar = tqdm(total=rosbag.get_message_count())

    for idx, row in obs.iterrows():
        pbar.update(1)
        image = bag_reader.find_closest_image(rosbag, img_dir, img_topic, row["timestamp"])
        cv_image.append(image)
    return cv_image






In [15]:
ROS_BAG_PATH = os.getcwd() + "/../rosbag/2024-09-10-17-09-22.bag"
IMG_DIR = os.getcwd() + "/../image_data"
OBS_TOPIC = "/left_arm/joint_states"
IMG_TOPIC = "/left_arm_camera/color/image_rect_color"
# Initialise bag reader
bag_reader = RosbagReader()
bag = bag_reader.get_rosbag(path_to_bag=ROS_BAG_PATH)
# Read and sync robot obs and image data
obs = read_observations(OBS_TOPIC, bag)
images = read_images(IMG_TOPIC, bag)
images.head()

  0%|          | 0/4052 [00:00<?, ?it/s]

100%|██████████| 155/155 [00:00<00:00, 361.96it/s]


,timestamp,cv::Mat
0,2024-09-10 17:09:22.994849280,"[[[170, 196, 206], [170, 196, 206], [169, 195,..."
1,2024-09-10 17:09:23.024444160,"[[[167, 189, 205], [167, 189, 205], [167, 189,..."
2,2024-09-10 17:09:23.056168192,"[[[167, 189, 205], [167, 189, 205], [167, 189,..."
3,2024-09-10 17:09:23.085401088,"[[[167, 189, 205], [167, 189, 205], [167, 189,..."
4,2024-09-10 17:09:23.125168128,"[[[167, 189, 200], [167, 189, 200], [167, 189,..."


In [20]:
!pip install imageio[ffmpeg]



     |████████████████████████████████| 315 kB 8.5 MB/s eta 0:00:01
     |████████████████████████████████| 4.4 MB 20.7 MB/s eta 0:00:01
     |████████████████████████████████| 26.9 MB 134.6 MB/s eta 0:00:01


In [24]:
print(type(cv_image))

<class 'list'>


In [27]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

matplotlib.rcParams['animation.embed_limit'] = 2**128

def play_image_sequence_in_notebook(cv_image_list, fps=30):
    """
    Display a sequence of OpenCV images as a video in a Jupyter Notebook.
    
    Args:
        cv_image_list (list): List of OpenCV images to display.
        fps (int): Frames per second (FPS) to simulate video playback.
    """
    fig, ax = plt.subplots()

    # Initialize the plot with the first image
    im = ax.imshow(cv_image_list[0])

    # Function to update the plot for each frame
    def update_frame(i):
        im.set_data(cv_image_list[i])
        return [im]

    # Create the animation
    frame_interval = int(1000 / fps)  # Interval in milliseconds between frames
    ani = animation.FuncAnimation(fig, update_frame, frames=len(cv_image_list), interval=frame_interval, blit=True)

    # Display the animation in the notebook
    return HTML(ani.to_jshtml())

# Example usage with your list of OpenCV images (cv_image_list)
# Assuming cv_image_list is a list of OpenCV images
play_image_sequence_in_notebook(cv_image, fps=30)
